In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder

#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')

import nltk
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import sys
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.metrics import accuracy_score, log_loss
import random
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    naive_bayes.MultinomialNB(),
    KNeighborsClassifier(2),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    #GaussianNB(),
    #LinearDiscriminantAnalysis(),
    #QuadraticDiscriminantAnalysis(),
]

log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

#Local imports
%load_ext autoreload
%autoreload 2
from unwiki import unwiki
import ner
import parsing_xml as px

In [4]:
# define Clean function to cleanse and standarize words
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalize

#prepare the dataset
allData = pd.DataFrame()
with open('../sample18/defs.txt','r') as f1:
    all_data_texts = f1.readlines()
all_data_labels = len(all_data_texts)*[1.0]
with open('../sample18/nondefs.txt', 'r') as f2:
    all_data_texts_rand = f2.readlines()
all_data_texts += all_data_texts_rand
all_data_labels += len(all_data_texts_rand)*[0.0]

# 1.0 will represent definitions is true 0.0 means it is false (not a definition)
allData['labels'] = all_data_labels
allData['texts'] = all_data_texts

# Split and randomize the datasets
train_x, test_x, train_y, test_y = model_selection.train_test_split(allData['texts'], allData['labels'])

# Vectorize all the paragraphs and definitions in the dataset
count_vect = CountVectorizer(analyzer='word', tokenizer=nltk.word_tokenize, ngram_range=(1,2))
count_vect.fit(allData['texts'])
xtrain = count_vect.transform(train_x)
xtest = count_vect.transform(test_x)

# Train Multinomial Naive Bayes model and print test metrics
#clf = naive_bayes.MultinomialNB().fit(xtrain, train_y)
#predictions = clf.predict(xtest)
#print(metrics.classification_report(predictions,test_y))

In [30]:
param_lst = [80,90,100,120]
SVC_lst = [SVC(kernel="rbf", C=param, probability=True) for param in param_lst]

In [ ]:
for C_param, clf in zip(param_lst,SVC_lst):
    name = clf.__class__.__name__
    clf.fit(xtrain, train_y)
    
    print("="*30)
    print(name, ',  C=', C_param)
    
    print('****Results****')
    predictions = clf.predict(xtest)
    acc = accuracy_score(test_y, predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    predictions = clf.predict_proba(xtest)
    ll = log_loss(test_y, predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

SVC ,  C= 80
****Results****
Accuracy: 84.6322%
Log Loss: 0.3347375121199043
SVC ,  C= 90
****Results****
Accuracy: 84.9229%
Log Loss: 0.33291331690653253
SVC ,  C= 100
****Results****
